In [ ]:
import numpy as np
from pymedphys import mosaiq
from pprint import pprint
with mosaiq.connect('.') as cursor:
    cursor.execute('SELECT SIT_SET_ID, Pat_ID1, Site_Name FROM Site WHERE Version = %s', str(0))
    sites = list(cursor)
    pprint(sites[19])

In [ ]:
from datetime import datetime, timedelta
from sklearn.cluster import AgglomerativeClustering

def cluster_sessions(tx_dttms, interval=timedelta(hours=3)):
    """
    """
    timestamps = [[tx_dttm.timestamp()] for tx_dttm in tx_dttms]
    clusterer = AgglomerativeClustering(n_clusters=None, distance_threshold=interval.seconds)
    labels = clusterer.fit_predict(timestamps)
    current_session_number, current_label = labels[0], 1    
    start_session = datetime.fromtimestamp(timestamps[0][0])
    end_session = start_session
    for label, timestamp in zip(labels, timestamps):
        if label != current_label:
            yield (current_session_number, start_session, end_session)
            current_session_number += 1            
            current_label = label
            start_session = datetime.fromtimestamp(timestamp[0])
            end_session = start_session
        else:
            end_session = datetime.fromtimestamp(timestamp[0])
    yield (current_session_number, start_session, end_session)            
        
test_dttms = [datetime.now() + timedelta(hours=h) for h in range(10)]

print('Date/times:')
for dttm in test_dttms:
    print('\t', str(dttm))

print('\nClustered Sessions:')
for session in cluster_sessions(test_dttms):
    print(f"\tSession#{session[0]}: {str(session[1])} to {str(session[2])}")

In [ ]:
def sessions_for_site(sit_set_id):
    """
    """
    with mosaiq.connect('.') as cursor:
        cursor.execute('SELECT Tx_DtTm FROM Dose_Hst ' \
                        'INNER JOIN Site ON Site.SIT_ID = Dose_Hst.SIT_ID ' \
                        'WHERE Site.SIT_SET_ID = %s ' \
                        'ORDER BY Dose_Hst.Tx_DtTm', sit_set_id)
        dose_hst_dttms = [rec[0] for rec in cursor]
        return cluster_sessions(dose_hst_dttms)

sit_set_id = 19
print(f"Dose_Hst.Tx_DtTm-based session intervals for SIT_SET_ID = {sit_set_id} in Msq db:")
for session in sessions_for_site(19):
    print(f"\tSession#{session[0]}: {str(session[1])} to {str(session[2])}")

In [ ]:
def session_offsets_for_site(sit_set_id):
    """
    """
    samples = []
    for session in sessions_for_site(sit_set_id):
        with mosaiq.connect('.') as cursor:
            start_time_str = (session[1] - timedelta(hours=1)).strftime('%Y-%m-%d %H:%M:%S')
            end_time_str = (session[2] + timedelta(hours=1)).strftime('%Y-%m-%d %H:%M:%S')
            print(start_time_str)
            print(end_time_str)
            cursor.execute('SELECT Superior_Offset, Anterior_Offset, Lateral_Offset FROM Offset ' \
                            'WHERE %s < Offset.Study_DtTm ' \
                            'AND Offset.Study_DtTm < %s ' \
                            'ORDER BY Offset.Study_DtTm', (start_time_str, end_time_str))
        
            # select offset
            offsets = list(cursor)
            if len(offsets) > 0:
                samples.append([session[0]] + list(offsets[0]))
    return np.array(samples)

print(session_offsets_for_site(19))

In [ ]:
def mean_session_offset_for_site(sit_set_id):
    offsets = session_offsets_for_site(sit_set_id)
    return np.mean(offsets, 0)
print(mean_session_offset_for_site(19))

In [ ]:
def localization_offset_for_site(connection, sit_set_id):
    """
    """
    cursor = connection.cursor(as_dict=True)
    cursor.execute("SELECT * " \
                   "FROM Offset " \
                   f"WHERE {session['start_session'].strftime()} <= Offset.Study_DtTm " \
                   f"AND Offset.SIT_SET_ID = {sit_set_id} ")    
    localization_offset = list(cursor)[0]
    test_offset = np.array([0.0, 0.0, 0.0])
    
    # determine session for localization offset
    return first_session, localization_offset

In [ ]:
def generate_trending_site_records(connection):
    cursor = connection.cursor(as_dict=True)
    cursor.execute('SELECT * FROM Site WHERE Version = 0')
    for site in cursor:
        pprint(site)
        sessions = sessions_for_site(connection, site)
        mean_offset, session_count = mean_offset_for_site(site, sessions)
        localization_offset = localization_offset_for_site(connection, site)
        yield { 'SIT_SET_ID':site,
                'Mean_Offset':mean_offset,
                'Session_Count':session_count,
                'Localization_Offset':localization_offset }

In [ ]:
# use the from_tesnor_slice to create a generator

In [ ]:
import pymedphys
with pymedphys.mosaiq.connect('.') as cursor:
    cursor.execute('SELECT * FROM Offset WHERE Version = 0')
    print(list(cursor))

In [ ]:
import trending_anomaly_model

In [ ]:
dir(trending_anomaly_model)